In [1]:
import os
import pandas as pd
import re
import csv
import subprocess
import numpy as np
import arff
import pickle
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef

analisysData = []

def weka_tokenizer(doc):
    delimiters_regexp = re.compile("[ |\n|\f|\r|\t|.|,|;|:|'|\"|(|)|?|!]")
    return list(filter(None, delimiters_regexp.split(doc)))

In [2]:
vocabularyFile = '../../RQ 1/RQ 1.1/vocabulary/MSR4FlakinessOriginal.dict'
testDataset = '../../datasets/flakies_rq_21.csv'

X_idFlakiesDataset = pd.read_csv(testDataset)
y_idFlakiesDataset = X_idFlakiesDataset['klass']

vocabulary = pickle.load(open(vocabularyFile, 'rb'))

vectorizer = CountVectorizer(analyzer='word', max_features=1500, tokenizer=weka_tokenizer, vocabulary=vocabulary)
bowToken = vectorizer.fit_transform(X_idFlakiesDataset['tokens'])
bowData = pd.DataFrame(bowToken.toarray(), columns=vectorizer.get_feature_names())

X_idFlakiesDataset.drop('tokens', axis=1, inplace=True)
X_idFlakiesDataset = X_idFlakiesDataset.join(bowData)
X_idFlakiesDataset.drop('klass', axis=1, inplace=True)
X_idFlakiesDataset.drop('project', axis=1, inplace=True)

# y_probs = classifier.predict_proba(X_idFlakiesDataset)[:,1]

In [3]:
classifiers = {}

for algSav in os.listdir("../../RQ 1/RQ 1.1/classifiers/"):
    classifiers[algSav.split('.')[0]] = pickle.load(open("../../RQ 1/RQ 1.1/classifiers/" + algSav, 'rb'))

In [4]:
print('Test dataset:', testDataset)
print('Vocabulary:', vocabularyFile)
print(len(y_idFlakiesDataset), "flakie samples")
print(len(vectorizer.vocabulary), "features \n")

for keyClassifier, classifier in classifiers.items():

    predict = classifier.predict(X_idFlakiesDataset)

    classificationReport = classification_report(y_idFlakiesDataset, predict, output_dict=True)['Flakey']
    classifierScore = classifier.score(X_idFlakiesDataset, y_idFlakiesDataset)      

    cmStr = confusion_matrix(y_idFlakiesDataset, predict)[0]

    cm = re.sub('\[|\]', '', str(cmStr))

    VP = cm.split(" ", 1)[0] 
    FN = cm.split(" ", 1)[1] 

    classifierResult = {
        'Validation Dataset': testDataset, 
        'Vocabulary': vocabularyFile, 
        'Classifier': keyClassifier, 
        'Precision': classificationReport['precision'],	
        'Recall': classificationReport['recall'], 
        'F1-Score': classificationReport['f1-score'], 
        'Support': classificationReport['support'],
        'Accuracy': classifierScore,
        # 'AUC': roc_auc_score(y_idFlakiesDataset, y_probs),
        'MCC': matthews_corrcoef(y_idFlakiesDataset, predict),
        'VP': VP,
        'FN': FN
    }

    print('   - ', keyClassifier, classifierResult)

    analisysData.append(classifierResult)

Test dataset: ../../datasets/flakies_rq_21.csv
Vocabulary: ../../RQ 1/RQ 1.1/vocabulary/MSR4FlakinessOriginal.dict
80 flakie samples
1500 features 

   -  naiveBayes {'Validation Dataset': '../../datasets/flakies_rq_21.csv', 'Vocabulary': '../../RQ 1/RQ 1.1/vocabulary/MSR4FlakinessOriginal.dict', 'Classifier': 'naiveBayes', 'Precision': 1.0, 'Recall': 0.1125, 'F1-Score': 0.20224719101123595, 'Support': 80, 'Accuracy': 0.1125, 'MCC': 0.0, 'VP': '', 'FN': '9 71'}
   -  knn {'Validation Dataset': '../../datasets/flakies_rq_21.csv', 'Vocabulary': '../../RQ 1/RQ 1.1/vocabulary/MSR4FlakinessOriginal.dict', 'Classifier': 'knn', 'Precision': 1.0, 'Recall': 0.1875, 'F1-Score': 0.3157894736842105, 'Support': 80, 'Accuracy': 0.1875, 'MCC': 0.0, 'VP': '15', 'FN': '65'}
   -  logisticRegression {'Validation Dataset': '../../datasets/flakies_rq_21.csv', 'Vocabulary': '../../RQ 1/RQ 1.1/vocabulary/MSR4FlakinessOriginal.dict', 'Classifier': 'logisticRegression', 'Precision': 1.0, 'Recall': 0.675, 'F1-

In [5]:
analisysDataCSV = pd.DataFrame(analisysData, columns=['Validation Dataset', 'Vocabulary', 'Training Dataset', 'Classifier', 'Precision',	'Recall', 'F1-Score', 'Support', 'Accuracy', 'AUC', 'MCC', 'VP', 'FN'])
analisysDataCSV.to_csv("execution_rq_21.csv", index=False)